# Previsão de preços na bolsa com RNN

## Importação Dados e Biblioteca

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [2]:
petr_treino = pd.read_csv('../datasets/acoes/PETR4_treino.csv')
petr_teste = pd.read_csv('../datasets/acoes/PETR4_teste.csv')

## Preprocessamento dos dados

In [3]:
petr_treino.dropna(inplace=True)
petr_teste.dropna(inplace=True)

In [4]:
petr_treino = petr_treino.loc[:,['Date', 'Open']]
petr_teste = petr_teste.loc[:,['Date', 'Open']]

In [5]:
normalizador_petr_treino = MinMaxScaler()
normalizador_petr_teste = MinMaxScaler()

In [6]:
petr_treino.loc[:, ['Open']] = normalizador_petr_treino\
                                .fit_transform(petr_treino.loc[:, ['Open']])
petr_teste.loc[:, ['Open']] = normalizador_petr_teste\
                                .fit_transform(petr_teste.loc[:, ['Open']])

In [8]:
previsores = []
preco_real = []

In [21]:
#Estebelecido min de 90 dados para prever
for counter in range(90, petr_treino.shape[0]):
    previsores.append(petr_treino.iloc[counter-90:counter,1])
    preco_real.append(petr_treino.iloc[counter,1])

In [33]:
previsores, preco_real = np.array(previsores), np.array(preco_real)

In [39]:
#Da Doc Keras inputs: A 3D tensor with shape [batch, timesteps, feature]
previsores = np.reshape(previsores, (previsores.shape[0], 
                                     previsores.shape[1],
                                     1))

## Construção da RNN

In [42]:
regressor = Sequential()

In [43]:
regressor.add(LSTM(units=100, return_sequences=True, input_shape=(previsores.shape[1],1)))
regressor.add(Dropout(0.3))

In [44]:
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units=50))
regressor.add(Dropout(0.3))

In [45]:
regressor.add(Dense(units=1, activation='linear'))

In [46]:
regressor.compile(optimizer = 'rmsprop', loss='mean_squared_error',
                  metrics = ['mean_absolute_error'])

In [47]:
regressor.fit(previsores, preco_real, epochs=100, batch_size=16)

Epoch 1/100
88/88 [==============================] - 10s 117ms/step - loss: 0.0264 - mean_absolute_error: 0.1199
Epoch 2/100
88/88 [==============================] - 11s 120ms/step - loss: 0.0125 - mean_absolute_error: 0.0841
Epoch 3/100
88/88 [==============================] - 9s 107ms/step - loss: 0.0091 - mean_absolute_error: 0.0717
Epoch 4/100
88/88 [==============================] - 10s 117ms/step - loss: 0.0076 - mean_absolute_error: 0.0639
Epoch 5/100
88/88 [==============================] - 12s 132ms/step - loss: 0.0067 - mean_absolute_error: 0.0620
Epoch 6/100
88/88 [==============================] - 10s 108ms/step - loss: 0.0061 - mean_absolute_error: 0.0574
Epoch 7/100
88/88 [==============================] - 9s 107ms/step - loss: 0.0054 - mean_absolute_error: 0.0552
Epoch 8/100
88/88 [==============================] - 9s 107ms/step - loss: 0.0047 - mean_absolute_error: 0.0517
Epoch 9/100
88/88 [==============================] - 10s 108ms/step - loss: 0.0046 - mean_absolute_

88/88 [==============================] - 9s 108ms/step - loss: 0.0014 - mean_absolute_error: 0.0270
Epoch 74/100
88/88 [==============================] - 9s 107ms/step - loss: 0.0015 - mean_absolute_error: 0.0278
Epoch 75/100
88/88 [==============================] - 9s 105ms/step - loss: 0.0014 - mean_absolute_error: 0.0269
Epoch 76/100
88/88 [==============================] - 10s 108ms/step - loss: 0.0014 - mean_absolute_error: 0.0274
Epoch 77/100
88/88 [==============================] - 9s 103ms/step - loss: 0.0013 - mean_absolute_error: 0.0271
Epoch 78/100
88/88 [==============================] - 10s 110ms/step - loss: 0.0015 - mean_absolute_error: 0.0280
Epoch 79/100
88/88 [==============================] - 9s 106ms/step - loss: 0.0014 - mean_absolute_error: 0.0269
Epoch 80/100
88/88 [==============================] - 9s 106ms/step - loss: 0.0013 - mean_absolute_error: 0.0263
Epoch 81/100
88/88 [==============================] - 10s 108ms/step - loss: 0.0014 - mean_absolute_error: 

## Testando o modelo

In [60]:
preco_real_teste = petr_teste.iloc[1:,1] 

In [77]:
previsores_teste = []
valor_teste = []

In [86]:
len(valor_teste)
len(previsores_teste)

156

In [79]:
#Estebelecido min de 90 dados para prever
for counter in range(90, preco_real_teste.shape[0]):
    previsores_teste.append(preco_real_teste[counter-90:counter])
    valor_teste.append(preco_real_teste[counter])

In [80]:
previsores_teste, valor_teste = np.array(previsores_teste), np.array(valor_teste)

In [81]:
#Da Doc Keras inputs: A 3D tensor with shape [batch, timesteps, feature]
previsores_teste = np.reshape(previsores_teste, (previsores_teste.shape[0], 
                                                 previsores_teste.shape[1],
                                                 1))

In [88]:
predicoes = regressor.predict(previsores_teste)

In [90]:
predicoes = normalizador_petr_teste.inverse_transform(predicoes)

In [93]:
predicoes.shape

(156, 1)

In [97]:
valor_teste = normalizador_petr_teste\
                .inverse_transform(np.reshape(valor_teste, (valor_teste.shape[0], 1)))

In [100]:
error = valor_teste - predicoes

In [104]:
mae = np.mean(np.abs(error))
mse = np.mean(np.abs(error)**2)

In [108]:
print(f"Erro absoluto médio: {mae} \n") 
print(f"Erro quadratico médio: {mse}" )

Erro absoluto médio: 0.3165668047039327 

Erro quadratico médio: 0.16245697630147063
